In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from proj1_helpers import *
from implementations import *
DATA_FOLDER = '../data/'

In [3]:
from split_data import split_data

In [4]:
y_train, x_train, id_train = load_csv_data(DATA_FOLDER + 'train.csv')
y_test, x_test, id_test = load_csv_data(DATA_FOLDER + 'test.csv')

In [5]:
y_train

array([ 1., -1., -1., ...,  1., -1., -1.])

In [6]:
np.shape(x_train)

(250000, 30)

In [7]:
id_train

array([100000, 100001, 100002, ..., 349997, 349998, 349999])

### Basic Least Squares

In [8]:
def compare_prediction(w_train, x, y):
    y_pred = predict_labels(w_train, x_train)
    return (y_pred == y_train).sum()/len(y_pred)

In [9]:
w_ls, loss_ls = least_squares(y_train, x_train)
np.shape(w_ls)

(30,)

In [10]:
compare_prediction(w_ls, x_train, y_train)

0.74432799999999999

### Stochastic Gradient Descent

In [11]:
np.shape(np.asarray([1]*30)[:,None].reshape(30, 1))

(30, 1)

In [12]:
np.shape(np.asarray([1]*30))

(30,)

In [13]:
initial_w = np.asarray([1]*30)
max_iters = 30
gamma = 0.7

In [14]:
w_sgd, loss_sgd = least_squares_SGD(y_train, x_train, initial_w, max_iters, gamma)

/Users/kevinkappel/Desktop/EPFL/Master1/ML/Isaac/ML_project/project1/template/loss.py:5: RuntimeWarning: overflow encountered in square
  return np.mean(e**2)/2.


In [15]:
compare_prediction(w_sgd, x_train, y_train)

0.63460799999999995

### Ridge Regression

In [19]:
def ridge_regression_demo(x, y, degree, ratio, seed):
    """ridge regression demo."""
    # define parameter
    lambdas = [3,4]#np.logspace(-5, 0, 15)
    # split data
    x_tr, y_tr, x_te, y_te = split_data(x, y, ratio, seed)
    
    loss = 1000
    w = 0
    
    for lambda_ in lambdas:
        # ridge regression
        weight, l = ridge_regression(y_tr, x_tr, lambda_)
        #weight = stack_w(weight, degree)
        mse_test = compute_loss(y_te, x_te, weight)
        
        if(mse_test < loss):
            loss = mse_test
            w = weight
        
    return w, loss

In [20]:
seed = 56
degree = 2
split_ratio = 0.9

In [21]:
w_r, loss_ridge = ridge_regression_demo(x_train, y_train, degree, split_ratio, seed)

In [22]:
compare_prediction(w_r, x_train, y_train)

0.74450799999999995

### Logistic Regression

In [57]:
def ridge_regression2(y, tx, lamb):
    """implement ridge regression."""
    aI = lamb * np.identity(tx.shape[1])
    a = tx.T.dot(tx) + aI
    b = tx.T.dot(y)
    return np.linalg.solve(a, b)

def ridge_regression3(y, tx, lambda_):
    first_term = tx.T.dot(tx)
    left = first_term + lambda_ *np.identity(tx.shape[1])
    right = tx.T.dot(y)
    w = np.linalg.solve(left, right)
    loss = compute_loss2(y, tx, w)
    return w, loss;

def compute_loss2(y, tx, w):
    e = y - tx.dot(w)
    return np.mean(e**2)/2.